In [1]:
import argparse
import concurrent.futures
import json
import os
import subprocess
import sys
from collections import defaultdict
from pathlib import Path
from typing import Any, Callable, Dict, List, Optional

import dash_core_components as dcc
import dash_html_components as html
import gin
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import sklearn
import sklearn.metrics
import tensorflow as tf
import tensorflow_probability as tfp
import yaml
from jupyter_dash import JupyterDash
from PIL import Image
from tqdm import tqdm

import lib_analysis
import lib_biased_mnist
import lib_plot
import lib_problem
import lib_toy

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

tqdm.pandas()
gin.enter_interactive_mode()

/home/ericpts/.local/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def read_df(data_root: Path, experiment_name: str) -> pd.DataFrame:
    df_path = data_root / experiment_name / "df.pickle"
    if df_path.exists():
        print(f"Found cached df at {df_path}. Reusing...", flush=True)
        return pd.read_pickle(str(df_path))

    DF = lib_analysis.read_problem(data_root, experiment_name)
    DF = lib_analysis.add_statistics_to_df(DF)

    DF.to_pickle(str(df_path), protocol=4)
    return DF

In [4]:
data_root = Path("../data/")
DF = read_df(data_root, "biased_mnist")

Found cached df at ../data/biased_mnist/df.pickle. Reusing...


In [5]:
def compute_ensemble_distribution(y_hat):
    m_1, m_2 = y_hat[:, 0, :], y_hat[:, 1, :]
    y_hat_ensemble = (m_1 + m_2) / 2
    return y_hat_ensemble


def pmax(y_hat_ensemble, _y_hat):
    return tf.math.reduce_max(y_hat_ensemble, axis=1)


def entropy(y_hat_ensemble, _y_hat):
    return -tf.math.reduce_sum(
        y_hat_ensemble * tf.math.log(y_hat_ensemble + tf.keras.backend.epsilon()),
        axis=1,
    )


def max_diff(y_hat_ensemble, y_hat):
    per_model = []
    for im in range(2):
        per_model.append(
            tf.math.reduce_max(tf.math.abs(y_hat_ensemble - y_hat[:, im, :]), axis=1)
        )
    ret = tf.math.reduce_max(per_model, axis=0)
    return ret


def average_diff(y_hat_ensemble, y_hat):
    per_model = []
    for im in range(2):
        per_model.append(
            tf.math.reduce_mean(tf.math.abs(y_hat_ensemble - y_hat[:, im, :]), axis=1)
        )
    ret = tf.math.reduce_mean(per_model, axis=0)
    return ret

In [6]:
def compute_auroc(in_y_hat, oo_y_hat, f_ensemble_score, higher_score_is_ood=True):
    id_score = f_ensemble_score(compute_ensemble_distribution(in_y_hat), in_y_hat)
    oo_score = f_ensemble_score(compute_ensemble_distribution(oo_y_hat), oo_y_hat)

    if higher_score_is_ood:
        id_label, oo_label = 0, 1
    else:
        id_label, oo_label = 1, 0

    y_true = tf.convert_to_tensor(
        ([id_label] * id_score.shape[0]) + ([oo_label] * oo_score.shape[0])
    )
    y_score = tf.concat([id_score, oo_score], axis=0)
    return sklearn.metrics.roc_auc_score(y_true, y_score)

In [7]:
def compute_metrics(in_y_hat, oo_y_hat):
    return {
        "auroc_pmax": compute_auroc(
            in_y_hat, oo_y_hat, pmax, higher_score_is_ood=False
        ),
        "auroc_entropy": compute_auroc(in_y_hat, oo_y_hat, entropy),
        "auroc_max_diff": compute_auroc(in_y_hat, oo_y_hat, max_diff),
        "auroc_average_diff": compute_auroc(in_y_hat, oo_y_hat, average_diff),
    }

In [8]:
def compute_metrics_for_row(row: pd.Series):
    gin.parse_config_file(row["gin_config_file"])

    problem = lib_biased_mnist.BiasedMnistProblem()
    in_dist = problem.generate_id_testing_data(include_bias=True)
    oo_dist = problem.generate_ood_testing_data(include_bias=True)

    models = []
    for p in row["model_paths"]:
        m = tf.keras.models.load_model(p, compile=False)
        models.append(m)

    in_X, in_y, in_y_biased, in_y_hat = lib_analysis.process_dataset(in_dist, models)
    oo_X, oo_y, oo_y_biased, oo_y_hat = lib_analysis.process_dataset(oo_dist, models)
    return compute_metrics(in_y_hat, oo_y_hat)

In [ ]:
DF = lib_analysis.add_columns_to_df(
    DF, DF.progress_apply(compute_metrics_for_row, axis=1)
)

In [ ]:
DF.to_pickle("/home/ericpts/work/data/biased_mnist/df.pickle")

In [10]:
top_level_tabs = []

for lr in np.unique(DF["Problem.initial_lr"]):
    print(f"Processing data for lr {lr}")
    tabs_for_outer = []
    for l_corr in np.unique(DF["label_correlation"]):
        df_for_fig = DF[
            (DF["label_correlation"] == l_corr) & (DF["Problem.initial_lr"] == lr)
        ].copy()

        figs = []
        for c in [
            "pmax",
            "entropy",
            "max_diff",
            "average_diff",
        ]:
            figs.append(
                lib_plot.end_to_end_plot(
                    go.Figure(),
                    df_for_fig,
                    f"auroc_{c}",
                    ["indep"],
                    f"AUROC({c}) l_corr {l_corr}; learning rate {lr}",
                )
            )

        tabs_for_outer.append(
            dcc.Tab(
                label=f"label correlation {l_corr}",
                children=[dcc.Graph(figure=fig) for fig in figs],
            )
        )
    top_level_tabs.append(
        dcc.Tab(label=f"learning rate {lr}", children=[dcc.Tabs(tabs_for_outer)])
    )


app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([dcc.Tabs(top_level_tabs)])
app.run_server(mode="inline")

Processing data for lr 0.0001
Processing data for lr 0.001
Processing data for lr 0.01
